Dans ce notebook nous importons les données de la Banque Mondiale avec l'API

In [8]:
import os
os.getcwd()

'/home/onyxia/work/Deforestation/Données'

In [5]:
## récupérer les données de l'API de la Banque Mondiale 
import requests
import pandas as pd

# fonction permettant de choisir l'indicateur et les années qu'on souhaite récupérer :

def fetch_worldbank_data(indicator, start_date="1960", end_date="2024"):
    
    base_url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}?date={start_date}:{end_date}&format=json"
    all_records = []
    page = 1

    url = f"{base_url}&page={page}"
    req = requests.get(url)
    data_eco = req.json()
# déterminer le nombre de pages non vides
    if len(data_eco) > 0 and isinstance(data_eco[0], dict) and "pages" in data_eco[0]:
        total_pages = data_eco[0]["pages"]
        print(f"{indicator} - Nombre total de pages : {total_pages}")
    else:
        print(f"{indicator} - Impossible de déterminer le nombre de pages.")
        return None
# faire tourner le code jusqu'à la dernière page
    while page <= total_pages:
        url = f"{base_url}&page={page}"
        req = requests.get(url)
        data_eco = req.json()

        if len(data_eco) > 1 and isinstance(data_eco[1], list):
            all_records.extend(data_eco[1])
        page += 1

    df = pd.DataFrame(all_records)

    if "country" in df.columns and "date" in df.columns and "value" in df.columns:
        # Extraire le nom du pays depuis le dictionnaire dans la colonne 'country'
        df["country"] = df["country"].apply(lambda x: x["value"] if isinstance(x, dict) else x)
        table = df[["date", "value", "country"]]
        return table
    else:
        print(f"{indicator} - Les colonnes attendues ne sont pas disponibles.")
        return None


data_BM= fetch_worldbank_data("NY.GDP.MKTP.CD", start_date="1960", end_date="2024")
data_BM=data_BM.rename(columns={"value" : "PIB"})

NY.GDP.MKTP.CD - Nombre total de pages : 341


Pour pouvoir merger les données de la banque mondiale avec celles des deux autres bases de données nous utilisons les codes pays avec le package pycountry pour obtenir les codes pays iso 3166.

In [6]:
import pycountry

# Fonction pour récupérer le code pays ISO 3166
def get_country_code(country_name):
    try:
        # Recherche dans pycountry
        country = pycountry.countries.lookup(country_name)
        return country.alpha_3  # Retourne le code à 3 lettres
    except LookupError:
        return None  # Si le pays n'est pas trouvé

# Ajouter une colonne 'code_pays' avec les codes ISO 3166
data_BM['code_pays'] = data_BM['country'].apply(get_country_code)

On va maintenant changer les valeur Nan en pour pouvoir faire les régressions plus facilement

In [8]:
# Remplacer toutes les valeurs NaN par 0
#data_BM = merged_data.apply(lambda col: col.fillna(0) if col.isnull().any() else col)
#data_BM

,date,PIB,country,GINI,Part terres agricoles,Part terres foret,code_pays
0,1960,0.000000e+00,Afghanistan,0.0,0.0,0.0,AFG
1,1960,2.421063e+10,Africa Eastern and Southern,0.0,0.0,0.0,0
2,1960,1.190495e+10,Africa Western and Central,0.0,0.0,0.0,0
3,1960,0.000000e+00,Albania,0.0,0.0,0.0,ALB
4,1960,2.723615e+09,Algeria,0.0,0.0,0.0,DZA
...,...,...,...,...,...,...,...
17019,2023,1.742080e+10,West Bank and Gaza,0.0,0.0,0.0,0
17020,2023,1.061717e+14,World,0.0,0.0,0.0,0
17021,2023,0.000000e+00,"Yemen, Rep.",0.0,0.0,0.0,0
17022,2023,2.757796e+10,Zambia,0.0,0.0,0.0,ZMB


In [9]:
# Export des données en csv
#data_BM.to_csv('data_BM.csv', index=False)
